# 정수 인코딩
- 단어에 정수를 부여하는 방법 중, 단어를 빈도 순서로 정렬하고 나서 빈도 수가 높은 순서대로 차례로 낮은 숫자부터 정수를 부여하는 방법을 실습해보자!
- 단어의 빈도 수가 적당하게 분포되도록 의도적으로 만든 텍스트 데이터로 실습해볼 것임.

In [63]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [64]:
raw_text = "A barber is a person. a barber is good person. a barber is huge person. he Knew A Secret! The Secret He Kept is huge secret. Huge secret. His barber kept his word. a barber kept his word. His barber kept his secret. But keeping and keeping such a huge secret to himself was driving the barber crazy. the barber went up a huge mountain."

# 문장 토큰화
sentences = sent_tokenize(raw_text)
print(sentences)

['A barber is a person.', 'a barber is good person.', 'a barber is huge person.', 'he Knew A Secret!', 'The Secret He Kept is huge secret.', 'Huge secret.', 'His barber kept his word.', 'a barber kept his word.', 'His barber kept his secret.', 'But keeping and keeping such a huge secret to himself was driving the barber crazy.', 'the barber went up a huge mountain.']


- 정제 작업, 정규화 작업을 병행하여 전처리 수행
  - 단어를 소문자화해서 단어의 개수를 통일시키고
  - 불용어와 단어 길이 <= 2인 경우 단어를 제외시킨다.
- 이처럼 텍스트를 수치화하하는 것은 본격적으로 자연어 처리 작업에 들어간다는 의미이므로, 단어가 텍스트일 때만 할 수 있는 최대한의 전처리를 끝내놓아야 한다.

In [71]:
# 단어의 개수를 세기 위한 vocab
vocab = {}
preprocessed_sentences = []
stop_words = set(stopwords.words('english'))

for sentence in sentences:
  tokenized_sentence = word_tokenize(sentence)
  # print(tokenized_sentence)

  result = []
  for word in tokenized_sentence:
    word = word.lower()
    if word not in stop_words:
      if len(word) > 2:
        result.append(word)
        if word not in vocab:
          vocab[word] = 0
        vocab[word] += 1
  preprocessed_sentences.append(result)
print(preprocessed_sentences)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


In [67]:
# 각 단어에 대한 빈도 수 출력
print(vocab)

{'barber': 8, 'person': 3, 'good': 1, 'huge': 5, 'knew': 1, 'secret': 6, 'kept': 4, 'word': 2, 'keeping': 2, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1}


In [56]:
print(vocab["word"])

2


In [57]:
vocab.items()

dict_items([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept', 4), ('word', 2), ('keeping', 2), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)])

In [58]:
vocab_sorted = sorted(vocab.items(), key=lambda x:x[1], reverse=True)
vocab_sorted

[('barber', 8),
 ('secret', 6),
 ('huge', 5),
 ('kept', 4),
 ('person', 3),
 ('word', 2),
 ('keeping', 2),
 ('good', 1),
 ('knew', 1),
 ('driving', 1),
 ('crazy', 1),
 ('went', 1),
 ('mountain', 1)]

In [59]:
# 높은 빈도 수를 가진 단어일수록 낮은 정수를 부여한다.
# 내림차순 정렬이 되어있는 상태니까 처음 item부터 1부터 정수 부여
word_to_index = {}
i = 0
for (word, freq) in vocab_sorted:
  # 빈도 수 1 제외
  if freq > 1:
    i = i + 1
    word_to_index[word] = i
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7}


In [60]:
# 빈도 수 상위 5개의 단어만을 사용하고 싶은 경우
vocab_size = 5

# 인덱스가 5 초과인 단어 제거 과정
words_frequency = [word for word, index in word_to_index.items() if index > vocab_size]

for w in words_frequency:
  del word_to_index[w]
print(word_to_index)



{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


- word_to_index에는 빈도 수 상위 5개의 단어들만 저장되어 있다.
- word_to_index를 사용해서, 단어 토큰화가 된 상태로 저장된 sentences에 있는 각 단어를 정수로 바꾼다.
### 문제점
-  ['barber', 'good', 'person']에는 더 이상 word_to_index에는 존재하지 않는 단어인 'good'이라는 단어가 있다.
  - => Out-Of-Vocabulary(단어 집합에 없는 단어) 문제('OOV 문제')
- word_to_index에 'OOV'라는 단어를 새롭게 추가하고, 단어 집합에 없는 단어들은 'OOV'의 인덱스로 인코딩해라!

In [61]:
word_to_index['OOV'] = len(word_to_index) + 1
print(word_to_index)


{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'OOV': 6}


In [78]:
encoded_sentences = []
# print(preprocessed_sentences)
for sentence in preprocessed_sentences:
    encoded_sentence = []
    for word in sentence:
        if word in word_to_index.keys():
            encoded_sentence.append(word_to_index[word])
        else:
            encoded_sentence.append(word_to_index['OOV'])
    encoded_sentences.append(encoded_sentence)
print(encoded_sentences)

# comment; 아래 방식 구현도 가능하다.
# try:
#     # 단어 집합에 있는 단어라면 해당 단어의 정수를 리턴.
#     encoded_sentence.append(word_to_index[word])
# except KeyError:
#     # 만약 단어 집합에 없는 단어라면 'OOV'의 정수를 리턴.
#     encoded_sentence.append(word_to_index['OOV'])


[[1, 5], [1, 6, 5], [1, 3, 5], [6, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [6, 6, 3, 2, 6, 1, 6], [1, 6, 3, 6]]


- 지금까지 dict 자료형을 사용해서 정수 인코딩을 해봤다.
- 좀 더 쉽게 하도록 Counter모듈, NLTK의 FreqDist, Keras 토크나이저를 이용해보자

In [79]:
from collections import Counter

print(preprocessed_sentences)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


In [80]:
# 단어 집합을 만들기 위해 하나의 리스트로 만들기
all_words_list = sum(preprocessed_sentences, [])
print(all_words_list)

['barber', 'person', 'barber', 'good', 'person', 'barber', 'huge', 'person', 'knew', 'secret', 'secret', 'kept', 'huge', 'secret', 'huge', 'secret', 'barber', 'kept', 'word', 'barber', 'kept', 'word', 'barber', 'kept', 'secret', 'keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy', 'barber', 'went', 'huge', 'mountain']


In [81]:
# Counter 모듈을 이용해서 단어의 빈도 수 카운트
vocab = Counter(all_words_list)
print(vocab)

Counter({'barber': 8, 'secret': 6, 'huge': 5, 'kept': 4, 'person': 3, 'word': 2, 'keeping': 2, 'good': 1, 'knew': 1, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1})


In [82]:
print(vocab['secret'])

6


In [83]:
# most_common()을 통해 빈도 수가 높은 단어 리턴
vocab_size = 5
vocab = vocab.most_common(vocab_size)
vocab

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]

In [88]:
# 이전과 같은 방식으로 높은 빈도 수를 가진 단어일수록 낮은 정수 인덱스를 부여한다.

word_to_index = {}
i = 1
for key, value in vocab:
  word_to_index[key] = i
  i += 1
word_to_index

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}

In [89]:
# FreqDist 사용해보기. Counter()과 같은 방법으로 사용할 수 있음

from nltk import FreqDist
import numpy as np

In [95]:
# np.hstack으로 문장 구분을 제거함
print(preprocessed_sentences)
print(np.hstack(preprocessed_sentences))

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]
['barber' 'person' 'barber' 'good' 'person' 'barber' 'huge' 'person'
 'knew' 'secret' 'secret' 'kept' 'huge' 'secret' 'huge' 'secret' 'barber'
 'kept' 'word' 'barber' 'kept' 'word' 'barber' 'kept' 'secret' 'keeping'
 'keeping' 'huge' 'secret' 'driving' 'barber' 'crazy' 'barber' 'went'
 'huge' 'mountain']


In [96]:
vocab = FreqDist(np.hstack(preprocessed_sentences))
print(vocab['barber'])

8


In [97]:
vocab_size = 5
vocab = vocab.most_common(vocab_size)
print(vocab)

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]


In [98]:
# enumerate()사용하기
word_to_index = {word[0] : index + 1 for index, word in enumerate(vocab)}
word_to_index

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}

In [100]:
# enumerate 이해하기
test_input = ['a', 'b', 'c', 'd', 'e']
for idx, val in enumerate(test_input):
  print(idx, val)
  print("value: {}, index: {}".format(val, idx))

0 a
value: a, index: 0
1 b
value: b, index: 1
2 c
value: c, index: 2
3 d
value: d, index: 3
4 e
value: e, index: 4


In [103]:
# Keras의 텍스트 전처리
# 정수 인코딩을 위해 케라스의 전처리 도구인 토크나이저를 사용하기도 함.

from tensorflow.keras.preprocessing.text import Tokenizer

preprocessed_sentences

[['barber', 'person'],
 ['barber', 'good', 'person'],
 ['barber', 'huge', 'person'],
 ['knew', 'secret'],
 ['secret', 'kept', 'huge', 'secret'],
 ['huge', 'secret'],
 ['barber', 'kept', 'word'],
 ['barber', 'kept', 'word'],
 ['barber', 'kept', 'secret'],
 ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'],
 ['barber', 'went', 'huge', 'mountain']]

In [104]:
tokenizer = Tokenizer()

# fit_on_texts()안에 코퍼스를 입력으로 넣으면 빈도 수를 기준으로 단어 집합을 생성한다! -> 앞에서 한 과정을 이 함수 하나를 통해 할 수 있는 것임!
tokenizer.fit_on_texts(preprocessed_sentences)

In [108]:
print(tokenizer.word_index) # 각 단어에 부여된 인덱스 확인
print(tokenizer.word_counts) # 각 단어 수 확인

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7, 'good': 8, 'knew': 9, 'driving': 10, 'crazy': 11, 'went': 12, 'mountain': 13}
OrderedDict([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept', 4), ('word', 2), ('keeping', 2), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)])


In [109]:
# 코퍼스의 단어를 정해진 인덱스로 반환하기
print(tokenizer.texts_to_sequences(preprocessed_sentences))

[[1, 5], [1, 8, 5], [1, 3, 5], [9, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [7, 7, 3, 2, 10, 1, 11], [1, 12, 3, 13]]


In [110]:
vocab_size = 5
tokenizer = Tokenizer(num_words = vocab_size + 1) # 상위 5개 단어만 사용
tokenizer.fit_on_texts(preprocessed_sentences)
# 이렇게 건 개수 제한 조건은 texts_to_sequences를 사용할 때 적용됨.

In [113]:
# 케.토는 기본적으로 OOV가 나오면 단어->정수 과정에서 아예 그 단어를 제거함
# 이를 OOV로서 보존하고 싶다면 Tokenizer의 인자 oov_token을 사용해야 함

vocab_size = 5
# 숫자 0, oov를 고려해서 단어 집합의 크기에 +2 해줌
tokenizer = Tokenizer(num_words=vocab_size+2, oov_token='OOV')
tokenizer.fit_on_texts(preprocessed_sentences)

print(f"단어 OOV의 인덱스 : {tokenizer.word_index['OOV']}")
# 케라스 토크나이저는 기본적으로 oov_token의 인덱스를 1로 합니다.

단어 OOV의 인덱스 : 1


In [114]:
# 이제 정수 인코딩 진행
print(tokenizer.texts_to_sequences(preprocessed_sentences))

[[2, 6], [2, 1, 6], [2, 4, 6], [1, 3], [3, 5, 4, 3], [4, 3], [2, 5, 1], [2, 5, 1], [2, 5, 3], [1, 1, 4, 3, 1, 2, 1], [2, 1, 4, 1]]


- 빈도 수 상위 5개의 단어는 2~6의 인덱스이고
- 그 외 단어 집합에 없는 'good'같은 단어들은 전부 'OOV'의 인덱스인 1로 인코딩되었다.